In [273]:
import math
import copy

In [274]:
# helper input processing functions

DIGITS = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

# assume that for all equations the variable order is the same
# assume that a scalar is always concatenated with the variable
# assume that between terms there is a space, terms = {variables, operands}
# assume no missing terms: 2x + 0y = 5 is the only valid way
# valid example: x + 2y = 5
# invalid example: 1 * x= 1 (two assumptions violated)

def read_number(line, idx):
    """
    returns the input string scalar as an integer and the start index of the variable
    """
    number = ""
    last_digit_idx = 0
    line_len = len(line)
    for i, c in enumerate(line, idx):
        if i >= line_len:
            break
        c = line[i]
        if c == '-' or c == '+' or f"{c}" in DIGITS:
            number += f"{c}"
        else:
            last_digit_idx = i
            break
    
    if number == "" or number == "+":
        number = "1"
    if number == "-":
        number = "-1"
    return int(number), last_digit_idx

def read_variable(line, idx):
    """
    return the index past the variable
    """
    last_letter_idx = idx
    for i, c in enumerate(line, idx):
        c = line[i] # is this a bug? ...
        if f"{c}" in DIGITS or f"{c}" == ' ':
            last_letter_idx = i
            break
    return last_letter_idx

In [275]:
# 1)

input_path = 'input.txt'

# assume that the count of variables and the one of the equations is equal to 3
count_var = 3
A = [[0 for _ in range(count_var)] for _ in range(count_var)]
B = [0 for _ in range(count_var)]
# no need of actually storing the variable names, it is enough to index them
X = [x for x in range(count_var)]

with open(input_path, "r") as file:
    for idx, input_line in enumerate(file):
        next_idx = 0
        line_len = len(input_line)
        
        # a while iteration reads the scalar, the variable, the next operand
        # previous operation registered
        # for "2x + 5y = 1", when reading 5y, the previous operand will be 1(+)
        operand = 1

        # read variables with their scalars until the equal sign is visible
        matrix_idx = 0
        while next_idx < line_len:
            number, next_idx = read_number(input_line, next_idx)
            next_idx = read_variable(input_line, next_idx)
            
            # add the read number to the matrix
            number *= operand
            A[idx][matrix_idx] = number
            matrix_idx += 1

            # determine the operand
            next_idx += 1
            operand = 1
            if input_line[next_idx] == '-':
                operand = -1
            if input_line[next_idx] == '=':
                break
            next_idx += 2
        
        # read the last number after the equal sign
        next_idx += 2
        number, _ = read_number(input_line, next_idx)
        # add it to the B array
        B[idx] = number

print("A:\n", A)
print("B:\n", B)
print("X:\n", X)

A:
 [[2, 3, -1], [1, -1, 4], [3, 1, 2]]
B:
 [5, 6, 7]
X:
 [0, 1, 2]


In [276]:
# 2)

# asumme a square matrix for the below operations

def determinant(matrix):
    n = len(matrix)
    sum = 0
    for aux in range(n):
        add = 1
        sub = 1
        for i in range(n):
            add *= matrix[(aux + i) % n][i]
            sub *= matrix[(aux + i) % n][n - 1 - i]
        sum += add
        sum -= sub
    return sum

def trace(matrix):
    n = len(matrix)
    sum = 0
    for i in range(n):
        sum += matrix[i][i]
    return sum

def norm(vector):
    sum = 0
    for value in vector:
        sum += value * value
    return math.sqrt(sum)

def transpose(matrix):
    n = len(matrix)
    for i in range(n):
        for j in range(i + 1, n):
            temp = matrix[j][i]
            matrix[j][i] = matrix[i][j]
            matrix[i][j] = temp
    return matrix

# assume square matrices of the same size
def multiply(matrix0, matrix1):
    n = len(matrix0)
    multiplication = [[0 for _ in range(n)] for _ in range(n)]
    
    for i in range(n):
        for j in range(n):
            # i is index of the row of matrix0
            # j is index of the column of matrix1
            for k in range(n):
                multiplication[i][j] += matrix0[i][k] * matrix1[k][j]
    return multiplication

print(determinant(A))
print(trace(A))
print(norm(B))
print(transpose([[1, 2], [3, 4]]))
print(multiply(A, A))

14
3
10.488088481701515
[[1, 3], [2, 4]]
[[4, 2, 8], [13, 8, 3], [13, 10, 5]]


In [277]:
# 3)

def cramer(A, B, idx):
    n = len(B)
    A_copy = copy.deepcopy(A)
    for i in range(n):
        A_copy[i][idx] = B[i]
    return determinant(A_copy) / determinant(A)

solution = [0.0, 0.0, 0.0]
for i in range(len(A)):
    solution[i] = cramer(A, B, i)

print(solution)

[0.35714285714285715, 2.0714285714285716, 1.9285714285714286]


In [ ]:
# 4)